# Explaining Knowledge Graphs with LLM

In [1]:
%%capture

! CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.2.26
! wget https://huggingface.co/TheBloke/Llama-2-7B-Chat-GGUF/resolve/main/llama-2-7b-chat.Q4_K_M.gguf?download=true
! mv llama-2-7b-chat.Q4_K_M.gguf?download=true ./llama-2-7b-chat.Q4_K_M.gguf
! pip install rdflib==7.0.0

## Main class for running LLama

In [2]:
from llama_cpp import Llama
import os
import re
from rdflib import Graph, Namespace, URIRef, Literal, BNode
from rdflib.namespace import RDFS, RDF, OWL, XSD, SOSA, SKOS
from copy import deepcopy
from collections import defaultdict
from itertools import product

In [3]:
model_path_middle = "./llama-2-7b-chat.Q4_K_M.gguf"
DEFAULT_ONTOLOGIES = ['sosa.ttl', 'rdfs.ttl', 'rdf.ttl', 'owl.ttl']
DEFAULT_IRIS = ['http://www.w3.org/ns/sosa', 'http://www.w3.org/2000/01/rdf-schema#', 'http://www.w3.org/1999/02/22-rdf-syntax-ns#', 'http://www.w3.org/2002/07/owl#']
get_iri = lambda x: '/'.join(x.split('/')[:-1])

def get_graph_data(graph):
  graph_data = graph.serialize()
  graph_data = re.sub("<[^ ]*\/([^ \/]*)>", "\g<1>", graph_data)
  graph_data = '\n'.join([x for x in graph_data.split('\n') if x[:7] != '@prefix' and x != ''])
  return graph_data

class GraphExplainer:
  def __init__(self, model_path=model_path_middle, n_ctx=4096, n_batch=2048):
    self.model_path = model_path
    self.n_threads = os.cpu_count()
    self.n_gpu = -1
    self.n_ctx = n_ctx
    self.n_batch = n_batch
    self.llm = self.load_model(n_ctx=self.n_ctx, n_batch=self.n_batch)

  def load_model(self, *args, **kwargs):
    return Llama(model_path=self.model_path, n_gpu_layers=self.n_gpu, n_threads=self.n_threads, *args, **kwargs)

  def explain_graph(self, system_prompt, user_prompt, *args, **kwargs):
    out = self.llm.create_chat_completion(
      messages = [
          {"role": "system", "content": system_prompt},
          {"role": "user", "content": user_prompt}
      ],
      *args, **kwargs
    )
    return out['choices'][0]['message']['content']

  def set_ontologies_lookup(self, ontologies_paths, ontologies_iris):
    if type(ontologies_paths) == str:
      ontologies_paths = [ontologies_paths]
    if type(ontologies_iris) == str:
      ontologies_iris = [ontologies_iris]
    all_ontologies = deepcopy(DEFAULT_ONTOLOGIES)
    all_ontologies.extend(ontologies_paths)
    all_iris = deepcopy(DEFAULT_IRIS)
    all_iris.extend(ontologies_iris)

    self.ontologies_dict = {}
    for ont, iri in zip(all_ontologies, all_iris):
      ont_graph = Graph().parse(ont, format='ttl')
      self.ontologies_dict[iri]=ont_graph
    return

  def process_graph(self, file_path):
    processed_graph = Graph().parse(file_path, format='ttl')
    required_ontologies = []
    for s, p, o in processed_graph.triples((None, None, None)):
        for element in [s,p,o]:
          if 'http' in str(element):
            required_ontologies.append(element)
    required_ontologies = list(set(required_ontologies))

    ontologies_graph = Graph()
    for data_required in required_ontologies:
      temp_iri = get_iri(data_required)
      if temp_iri in self.ontologies_dict.keys():
        relevant_ontology = self.ontologies_dict[temp_iri]
        for s, p, o in relevant_ontology.triples((data_required, None, None)):
          ontologies_graph.add((s,p,o))
    ontology_text = get_graph_data(ontologies_graph)
    input_text = get_graph_data(processed_graph)
    return input_text, ontology_text

## Running the code

In [4]:
explainer = GraphExplainer()
explainer.set_ontologies_lookup(['./content/pilot2.ttl'], ['https://assist-iot.eu/ontologies'])

AVX = 1 | AVX2 = 1 | AVX512 = 1 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


In [8]:
tags = ['tags_0000003596.ttl', 'tags_0000005028.ttl', 'tags_0000014000.ttl', 'tags_0000014017.ttl', 'tags_0000014019.ttl', 'tags_0000014025.ttl', 'tags_0000014026.ttl', 'tags_0000014036.ttl']

# Results for tags

In [6]:
system_prompt_1 = lambda x: """
You are a master in the field of knowledge graphs. You perfectly know turtle and you explain their contents using layman's terms. Use the following ontology to understand the data.
""" + x
input_prompt_1 = lambda x: "Summarize only the data of the following graph, in sentences only. Focus on the values at the end of each line.\n\n" + x

In [9]:
for tag in tags:
  print(tag)
  input_text, ontology_text = explainer.process_graph(tag)
  print(explainer.explain_graph(system_prompt_1(ontology_text), input_prompt_1(input_text)))
  print()

tags_0000003596.ttl
  Here are the values from the given graph:
* 33298 is a Platform that hosts a Sensor.
* The Sensor made an Observation with a result of -4 for X, -3 for Y, and 256 for Z, measured at time "2023-07-03T10:43:28.428205"^^xsd:dateTime.
* The same Sensor made another Observation with a result of -4 for X, -8 for Y, and 257 for Z, measured at time "2023-07-03T10:43:29.428205"^^xsd:dateTime.
* The Platform has an Acceleration result with acceleration window number 2, which has X value -4, Y value -3, and Z value 256.
* The Platform has a Tag metadata result with alarm false, watch battery level 90, and watch connected true.
* The BIM location result has accuracy value 100, location measurement type measurementNone, X value 0, Y value 0, and Z value 0, measured at time "2023-07-03T10:43:28.428205"^^xsd:dateTime.

tags_0000005028.ttl


Llama.generate: prefix-match hit


  * The platform 33298 hosts the sensor with ID 33298.
* The sensor made an observation with a result of acceleration (Measure) with acceleration window number 2 and X value -2, Y value -7, and Z value 259.
* The sensor made another observation with a result of acceleration (Measure) with acceleration window number 1 and X value -3, Y value -7, and Z value 257.
* The Tag metadata result has an alarm set to false, watch battery level of 90, and watch connected status of true.
* The BIM location result has an accuracy value of 1 millimetre, location measurement type of none, X value of 0, Y value of 0, and Z value of 0.
* The observation result has a result time of "2023-07-03T11:09:12.681948"^^xsd:dateTime.

tags_0000014000.ttl


Llama.generate: prefix-match hit


  The data in the graph represents a set of observations, sensors, and results related to a BIM model. The observations are made by sensors, which host the platforms, and the results are stored in the hasResult property. The observations are linked to a result using the hasResult property, and the result is hosted by the platform. The data includes acceleration measurements with different windows (1 or 2), X, Y, and Z values, and the watch battery level and connection status of the sensor. Additionally, there are location measurements with an accuracy value and a measurement type. The data is stored in a ontology-based data model (OBDM) and uses RDF terms to define the relationships between the entities.

tags_0000014017.ttl


...

  * The graph contains 1 Platform, 1 Sensor, and 3 Results.
* The Platform is hosted by the Sensor (7449).
* The Sensor made an Observation with a result of -252 for acceleration in the x-axis, 2 for acceleration in the y-axis, and -44 for acceleration in the z-axis.
* The first Result is an AccelerationResult with an acceleration window of 2 and a value of -253 for the x-axis, 8 for the y-axis, and -53 for the z-axis.
* The second Result is also an AccelerationResult with an acceleration window of 1 and a value of -253 for the x-axis, 8 for the y-axis, and -53 for the z-axis.
* The third Result is a TagMetadataResult with an alarm set to false, a watch battery level of 97, and a watch connected status of true.
* The LocationResult has a BIM location, an accuracy value of 0.01666668 for the x-axis, 0.038304687 for the y-axis, and a location measurement type of measurement2D. The x-value is 4204, the y-value is -17579, and the z-value is 344.
* The Observation time is July 4, 2023, at 8

Llama.generate: prefix-match hit


  * The graph contains 7449 Platforms, each with a sensor and multiple observations.
* Each observation has a result, which is an acceleration measurement in the tag acceleration unit (ns2:tagAccelerationUnit).
* The accelerations range from -251 to 8 G, with some values having a specific timestamp.
* Some observations have a result time, indicating when the observation was made.
* The graph also contains location measurements, which are classified as BIM locations (ns2:BIMLocation) and have an accuracy value in millimeters.
* The location measurements have a X, Y, and Z value, also in millimeters.
* Some observations have a watch battery level and watch connected status.
* The graph does not contain any alarm or watch battery level values.

tags_0000014036.ttl


Llama.generate: prefix-match hit


  * The graph represents a set of observations, sensors, and results related to the measurement of acceleration in a BIM model.
* The observations are made by sensors (e.g., 7449) and result in a set of measurements (e.g., acceleration values) for different axes (x, y, and z).
* Each observation has a unique identifier (e.g., "tag-7449") and is associated with a sensor that made the observation.
* The sensors are hosted by a platform (e.g., 7449), which provides the necessary infrastructure for the sensors to operate.
* Each result is associated with an observation and represents the outcome of the measurement process. The results include acceleration values for different axes (x, y, and z) and are represented as numerical values.
* The acceleration values are measured in units of millimeters per second squared (e.g., -333, -51, -82).
* Some of the results also include additional metadata, such as the alarm status (false), watch battery level (97), and watch connection status (true).
*

In [16]:
system_prompt_2 = lambda x: f"""
You are an expert in knowledge graphs. Use the following ontology to understand the data.

{x}

You are the assistant who, given a graph, can extract the most important information from it. Focus on the values at the end of each line.
Your answers should be clear, coherent and describe the most important facts recorded in the graph.
Please return full sentences, bullet points and focus on creating a human-readible text, and do not include class and property labels and names.
In summary, you should put focus on anomalies and potentially dangerous measurements and clearly inform the user about them.
"""
input_prompt_2 = lambda x: "Summarise the data in the following graph.\n\n" + x

In [13]:
for tag in tags:
  print(tag)
  input_text, ontology_text = explainer.process_graph(tag)
  print(explainer.explain_graph(system_prompt_2(ontology_text), input_prompt_2(input_text)))
  print()

tags_0000003596.ttl


Llama.generate: prefix-match hit


  The graph represents a set of observations, sensors, and results related to the measurement of acceleration, location, and battery level of a device. Here are the most important facts extracted from the graph:
* There are three observations (33298, 33299, and 33300) made by different sensors (sensor 1, sensor 2, and sensor 3).
* The observations were made at different times (July 3rd, 2023 at 10:43:28.428205 UTC) and have corresponding results.
* The results include acceleration measurements in three axes (x, y, and z) with different window numbers (2 and 1).
* The acceleration values range from -4 to 257 for window 2, and from -8 to 257 for window 1.
* The location of the device is measured using a BIM location sensor, which provides an accuracy value of 90 millimeters.
* The battery level of the device is 90%.
* There are no alarm or watch connected to the device.
* The device is hosted by platform 33298.

In summary, the graph shows that there are multiple observations and results

Llama.generate: prefix-match hit


  The graph represents a set of observations, sensors, and results related to the measurement of acceleration, location, and battery level of a device. Here are the most important facts extracted from the graph:
* There are three observations (33298, 33299, and 33300) made by different sensors (Sensor 1, Sensor 2, and Sensor 3).
* The observations were made at different times (July 3rd, 2023 at 11:09:12.681948 UTC) and have corresponding results.
* The results include acceleration measurements in three axes (x, y, and z) and location information within a BIM model.
* The acceleration measurements show anomalies and potentially dangerous values, such as -3 g in the x axis for observation 33298 at time 1, and -7 g in the y axis for observation 33299 at time 2. These values are marked with a comment "Anomaly!".
* The location measurements show that the device is located within a BIM model with an accuracy of 1 millimetre.
* The battery level of the device is 90% at time 1 for observation 

...

  The graph represents a set of observations, sensors, and results related to the measurement of acceleration in a BIM model. Here are the most important facts extracted from the graph:
* There are two observations with different acceleration windows (2 and 1) and numerical values for each window.
* The observation with acceleration window 2 has a negative value of -252 for the X axis, a positive value of 2 for the Y axis, and a negative value of -44 for the Z axis.
* The observation with acceleration window 1 has a negative value of -253 for the X axis, a positive value of 8 for the Y axis, and a negative value of -53 for the Z axis.
* There is one result that is a TagMetadataResult, which indicates that the measurement was made by a tag device. The result has an alarm status of false, a watch battery level of 97, and a watch connected status of true.
* There are two location measurements with different accuracy values (4204 millimeters for the X axis and -17579 millimeters for the Y 

Llama.generate: prefix-match hit


  The graph represents a set of observations, sensors, and results related to the acceleration of a tag within a BIM model. Here are some key points about the data:
* There are two observations with different acceleration windows (2 and 1) for the same tag.
* The observation with acceleration window 2 has an X value of -251, Y value of 8, and Z value of -51.
* The observation with acceleration window 1 has an X value of -251, Y value of 8, and Z value of -49.
* There is one result for each acceleration window, indicating the average acceleration over that time period.
* The results show a significant difference in acceleration between the two windows, with the second window showing a higher acceleration value.
* The tag's battery level is 97, and it is connected to the BIM model.
* There is no alarm detected.
Based on this data, it appears that there may be an issue with the tag's accuracy or calibration, as the acceleration values differ significantly between the two windows. Addition

Llama.generate: prefix-match hit


  Based on the provided graph, here are some key observations and potential issues:
1. Sensor 7449 is hosting another sensor (7449) and has made an observation with a result that includes acceleration data.
2. The acceleration data shows a significant deviation from the expected value in one of the windows (acceleration window number 2), which could indicate a potential issue or anomaly.
3. The observation also includes a result for the BIM location, which indicates the device's position within a BIM model.
4. The TagMetadataResult shows that the watch battery level is at 97% and the watch is connected.
5. There are two results for the measurement2D property, one with an X value of 4401 and one with an Y value of -17909. These values could indicate a potential issue or anomaly in the device's positioning.
6. The hasAlarm property is set to false, which could indicate that there are no alarm conditions detected.
7. The hasWatchBatteryLevel and hasWatchConnected properties show that the 

In [14]:
input_prompt_2_professor = lambda x: "I am a professor who specializes in knowledge graphs. Summarise the following graph focusing on its structure.\n\n" + x
for tag in tags:
  print(tag)
  input_text, ontology_text = explainer.process_graph(tag)
  print(explainer.explain_graph(system_prompt_2(ontology_text), input_prompt_2_professor(input_text)))
  print()

tags_0000003596.ttl


Llama.generate: prefix-match hit


  The graph provided contains the following information:
* A Platform (33298) that hosts a Sensor and makes Observations.
* A Sensor (33298) that makes Observations and has a result.
* An Observation with a result of -4 for X, -3 for Y, and 256 for Z, made by the Sensor at a specific time ("2023-07-03T10:43:28.428205"^^xsd:dateTime).
* Another Observation with a result of -4 for X, -8 for Y, and 257 for Z, made by the Sensor at a different time.
* A TagMetadataResult with an alarm set to false, watch battery level of 90, and watch connected status of true.
* A BIMLocation result with accuracy value measured in millimeters, location measurement type of "no location measurement", X value measured in millimeters, Y value measured in millimeters, and Z value measured in millimeters.
The graph shows that there are two Observations made by the Sensor at different times, one with a positive acceleration value and the other with a negative acceleration value. The TagMetadataResult indicates th

Llama.generate: prefix-match hit


  The graph provided contains the following classes and properties:
* sosa:Platform (with identifier "tag-33298")
	+ hosts 33298 (a Sensor)
* sosa:Sensor (with identifier "33298")
	+ madeObservation (an Observation with numerical value 0)
	+ hasResult (an Acceleration Result with acceleration window "2" and acceleration values -2 for X, -7 for Y, and 259 for Z)
	+ hasYValue (a Measure with unit ns1:tagAccelerationUnit and numerical value -7)
	+ hasZValue (a Measure with unit ns1:tagAccelerationUnit and numerical value 259)
* sosa:Result (with identifier "33298")
	+ AccelerationResult (with acceleration window "1" and acceleration values -3 for X, -7 for Y, and 257 for Z)
	+ TagMetadataResult (with alarm false, watch battery level 90, and watch connected true)
* aiot#LocationResult (with identifier "tag-33298")
	+ BIMLocation (with accuracy value [a Measure with unit ns2:millimetre and numerical value 0.038304687], location measurement type measurementNone, X value [a Measure with unit 

...

  The graph provided is a knowledge graph, which represents entities and their relationships in a structured manner. Here's a summary of the structure of the graph:
* Platform (7449):
	+ Hosts 7449 (a Sensor)
	+ Made observation (2 observations)
	+ Result (3 results, including 1 TagMetadataResult and 2 LocationResults)
	+ AccelerationWindow (2 acceleration windows)
* Sensor (7449):
	+ Hosts 7449 (a Platform)
	+ Made observation (1 observation)
	+ Result (1 result, including 1 TagMetadataResult and 1 LocationResult)
	+ AccelerationWindow (1 acceleration window)
* Observation:
	+ Has result (2 results, including 1 TagMetadataResult and 1 LocationResult)
	+ Made by Sensor (1 sensor)
	+ Result time (1 date-time value)
* TagMetadataResult:
	+ Has alarm (false)
	+ Has watch battery level (97)
	+ Has watch connected (true)
* LocationResult:
	+ Has accuracy value (1 measurement)
	+ Has location measurement type (2D)
	+ Has X value (1 measure)
	+ Has Y value (1 measure)
	+ Has Z value (1 measur

Llama.generate: prefix-match hit


  The graph provided contains the following information:
* A Platform (7449) that hosts a Sensor and an Observation.
* The Sensor (7449) has made two Observations, one with a result of -251 for X, 8 for Y, and -49 for Z, and another with a result of -251 for X, 8 for Y, and -51 for Z.
* The Observation (7449) has a result of 78 for the Acceleration measurement, with an acceleration window number of 2.
* The Platform (7449) has a Watch connected and the battery level is 97.
* The Observation (7449) has an Alarm set to false.
* The Location (7449) has a measurement type of 2D, with an accuracy value of 4164 for X, -17597 for Y, and 296 for Z.
The graph shows that there are two Observations made by the Sensor (7449), one with a positive result and another with a negative result. The Observation with the positive result has an acceleration value of 78, while the Observation with the negative result has an acceleration value of -251. The graph also shows that the Platform (7449) is connecte

Llama.generate: prefix-match hit


  The graph provided is a knowledge graph that represents a set of interconnected objects and their properties. Here are the key takeaways from the structure of the graph:
1. Platforms: There are two platforms (7449 and 7450) in the graph, which are connected to each other through the "hosts" relationship.
2. Sensors: There are three sensors (7449, 7450, and 7451) in the graph, which are connected to their respective platforms through the "hosts" relationship.
3. Observations: There are two observations (7449-observed and 7450-observed) in the graph, which are linked to their respective sensors through the "madeObservation" relationship.
4. Results: There are three results (7449-result, 7450-result, and 7451-result) in the graph, which are linked to their respective observations through the "hasResult" relationship.
5. Acceleration windows: There are two acceleration windows (window 2 and window 1) in the graph, which are linked to their respective results (7449-result and 7450-result)

In [15]:
input_prompt_2_supervisor = lambda x: "I am a supervisor at the contruction site. Report me about important incidents.\n\n" + input_text
for tag in tags:
  print(tag)
  input_text, ontology_text = explainer.process_graph(tag)
  print(explainer.explain_graph(system_prompt_2(ontology_text), input_prompt_2_supervisor(input_text)))
  print()

tags_0000003596.ttl


Llama.generate: prefix-match hit


  Based on the provided ontology, here are some important incidents and measurements extracted from the graph:
* The sensor "33298" is hosting a platform, which could indicate a potential safety hazard as it may be a device that is capable of emitting harmful radiation or other hazardous substances.
* The sensor "33298" has made an observation with an acceleration value of -4 in the x-axis, -3 in the y-axis, and 256 in the z-axis for acceleration window number 2. This could indicate a potential structural issue or instability in the construction site, as high accelerations can lead to damage to buildings or other structures.
* The sensor "33298" has made an observation with an acceleration value of -4 in the x-axis, -8 in the y-axis, and 257 in the z-axis for acceleration window number 1. This could indicate a potential safety hazard or structural issue similar to the one mentioned above.
* The result "33298" has a tag metadata result with an alarm set to false, watch battery level of 

Llama.generate: prefix-match hit


  Based on the provided ontology, here are some important incidents and measurements extracted from the graph:
1. Anomalous acceleration measurement: The sensor 33298 has made an observation with an accelerometer reading of -7 for the x-axis, -3 for the y-axis, and 259 for the z-axis. This is an anomaly compared to the normal range of values for these axes.
2. Watch battery level: The tag 33298 has a watch battery level of 90, which is above the minimum threshold of 70. This suggests that the tag may not be functioning properly.
3. Watch connected: The tag 33298 is connected to a device, indicating that it is transmitting data.
4. Location measurement type: The location measurement type for the tag 33298 is "measurementNone", which means that no location measurement was performed.
5. Acceleration unit: The acceleration unit for the tag 33298 is "tagAccelerationUnit", which has a numerical value of -2 for the x-axis, -7 for the y-axis, and 259 for the z-axis.
6. X, Y, and Z values: The 

...

  Based on the provided graph, here are some important incidents and potential safety concerns:
1. High acceleration reading: The result of Observation 1 has an accelerometer value of -252 for window 2, which is a high acceleration reading. This may indicate a potential safety concern, such as a structural failure or a sudden impact.
2. Low battery level: Tag 7449 has a watch battery level of 97, which is relatively low. This may indicate that the device needs to be replaced or recharged.
3. Watch connected: Tag 7449 is connected to a watch, which may indicate that the device is functioning properly and transmitting data correctly.
4. Location measurement type: The location measurement type for Tag 7449 is set to "2D", indicating that the device is capable of providing two-dimensional location measurements.
5. Acceleration unit: The acceleration unit for Tag 7449 is set to "tagAccelerationUnit", which may indicate a potential safety concern if the device is not functioning properly.
6.

Llama.generate: prefix-match hit


  Based on the provided graph, here are some important incidents that have occurred or potential issues that may arise:
* Anomaly in acceleration measurement: There is an unusual acceleration reading of -251 for observation ID 7449 in the second acceleration window. This could indicate a problem with the sensor or the measurement process.
* Dangerous battery level: The watch battery level for observation ID 7449 is at 97, which is relatively low and may require attention to ensure continued proper functioning of the device.
* Watch connected issue: The watch is connected for observation ID 7449, which could indicate a potential problem with the device or the measurement process.
* Location measurement issues: There are anomalies in the location measurements for observation ID 7449, particularly in the X and Z axes. This could indicate a problem with the location sensor or the measurement process.
* High acceleration reading: There is an unusual high acceleration reading of 8 for observ

Llama.generate: prefix-match hit


  Based on the provided graph, here are some important incidents and potential safety hazards that have been detected:
1. High acceleration reading: The observation 7449 has an accelerometer reading of -333 for the acceleration window "2". This is a potentially dangerous reading as it indicates a high level of vibration or movement, which could be indicative of structural damage or instability.
2. Low battery level: The tag 7449 has a watch battery level of 97, which is relatively low. This may indicate that the device is not functioning properly and may need to be replaced or recharged.
3. Discrepancy in location measurements: The location measurement for the tag 7449 indicates a discrepancy between the X, Y, and Z values. This could be indicative of a problem with the device's location sensors or a misalignment issue.
4. Alarm not triggered: The observation 7449 does not have an alarm triggered, which may indicate that the device is not functioning properly or that there is a problem